# GPX Viewer

This app lets you to display a track from a GPX file recorded with a GPS device.

In [20]:
import sys,os
from ipywidgets import Button, HTML, HBox, FileUpload, Label, Output, Layout, Image

sys.path.append(os.getcwd() + '/src')
import sailing_analytics
import trail_mapping

In [21]:
tools = ["voila", "ipyleaflet", "ipywidgets", "bqplot"]
logos = []
for tool in tools:
    with open(f'./img/{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='10px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [22]:
"""
create the output widget to place the results
"""
global out 
out = Output()

out_bottom = Output()

In [23]:
def show_uploader():
    uploader = FileUpload(accept='.gpx', multiple=False)

    def handle_upload(change):
        # keep only the last file
        # TODO: check if this should be fixed in FileUpload widget
        # when multiple=False
        *_, (_, f) = change['new'].items()
        gpx_content = f['content'].decode('utf-8')
        out.clear_output()
        with StringIO(gpx_content) as gpx_file:
            session = sailing_analytics.SailingSession(gpx_file)
            with out_bottom:
                show_map_polar_buttons(session)


    uploader.observe(handle_upload, names='value')
    display(uploader)

In [24]:
def show_examples():
    example_folder = "./examples"
    examples = [f for f in os.listdir(example_folder) if f.endswith('.gpx')]
    
    def create_example(name):
        filename = os.path.join(example_folder, name)
        
        @out.capture()
        def on_example_clicked(change):
            out.clear_output()
            with out:
                global session
                session = sailing_analytics.SailingSession(filename)
                show_map_polar_buttons(session)
                # trail_mapping.plot_gpx(f)
    
        button = Button(description=os.path.splitext(name)[0])
        button.on_click(on_example_clicked)
        return button
    
    buttons = [create_example(example) for example in examples]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)

In [25]:
def show_map_polar_buttons(session):
    def map_button():
        @out.capture()
        def map_button_clicked(change):
            out.clear_output()
            with out:
                session.map()

        button = Button(description="Show Map")
        button.on_click(map_button_clicked)
        return button

    def polar_button():
        @out.capture()
        def polar_button_clicked(change):
            session.polar()
            out.clear_output()
            # with out:
                # session.polar()

        button = Button(description="Show Polar")
        button.on_click(polar_button_clicked)
        return button

    buttons = [map_button(), polar_button()]
    line = HBox(buttons, layout=Layout(flex_flow='row', align_items='center'))
    display(line)


In [26]:
out

Output()

In [27]:
show_examples()

If you don't have a GPX file, try with one of the following examples:

In [28]:
show_uploader()

FileUpload(value={}, accept='.gpx', description='Upload')

In [29]:
out_bottom

Output()